# Project Overview 

In this project, I will be working with a data set of post submissions to popular technology website [Hacker News](https://news.ycombinator.com/). 

Hacker News is similar to reddit, and it is extremely popular in technology and startup circles. Posts that make it to the top of their listiings can get hundreds of thousands of views! 

# Project Goals 

Questions that I'd like to answer with this exploratory analysis are: 

* What type of posts receive more comments on average?
* Do posts created at certain times receive more engagement on average? 

# Exploring the data set

The data set was downloaded from [here](https://www.kaggle.com/hacker-news/hacker-news-posts). 

Some cleaning has already been performed on the data -- posts that did not receive any comments were removed, and of the remaining posts a random sample was chosen to form the data set. 

Here are the column definitions:

| column name  | column definition                                                                                                     |
|--------------|-----------------------------------------------------------------------------------------------------------------------|
| id           | The unique identifier from Hacker News for the post                                                                   |
| title        | The title of the post                                                                                                 |
| url          | The URL that the posts links to, if the post has a URL                                                                |
| num_points   | The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes |
| num_comments | The number of comments that were made on the post                                                                     |
| author       | The username of the person who submitted the post                                                                     |
| created_at   | The date and time at which the post was submitted                                                                     |

In [31]:
# reading in the data
import csv

f = open('hacker_news.csv')
hn = list(csv.reader(f))
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12579008',
  'You have two days to comment if you want stem cells to be classified as your own',
  'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018',
  '1',
  '0',
  'altstar',
  '9/26/2016 3:26'],
 ['12579005',
  'SQLAR  the SQLite Archiver',
  'https://www.sqlite.org/sqlar/doc/trunk/README.md',
  '1',
  '0',
  'blacksqr',
  '9/26/2016 3:24'],
 ['12578997',
  'What if we just printed a flatscreen television on the side of our boxes?',
  'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43',
  '1',
  '0',
  'pavel_lishin',
  '9/26/2016 3:19'],
 ['12578989',
  'algorithmic music',
  'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext',
  '1',
  '0',
  'poindontcare',
  '9/26/2016 3:16']]

In [32]:
headers = hn[0]
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [33]:
hn = hn[1:]
print(hn[:5])

[['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19'], ['12578989', 'algorithmic music', 'http://cacm.acm.org/magazines/2011/7/109891-algorithmic-composition/fulltext', '1', '0', 'poindontcare', '9/26/2016 3:16'], ['12578979', 'How the Data Vault Enables the Next-Gen Data Warehouse and Data Lake', 'https://www.talend.com/blog/2016/05/12/talend-and-Â\x93the-data-vaultÂ\x94', '1', '0', 'markgainor1', '9/26/2016 3:14']]


# Extracting posts

There are 2 types of posts that I am interested in: 

1. **Ask HN**, where users ask the Hacker News community questions such as "What is the best online course you've taken on Python for Data Science?" 

2. **Show HN**, where users submit posts showing the Hacker News community their projects, products, or just generally interesting things. 

In this next section, I will seperate the data into 2 lists - 1 list for 'Ask HN' and 1 list for 'Show HN'. This makes it easier to analyze the data later on.

In [78]:
# Identify posts that begin with 'Ask HN' or 'Show HN' and filter them into different lists

ask_posts = []
show_posts = []
other_posts = []

for row in hn: 
    title = row[1]
    title = title.lower()
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else: 
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

9139
10158
273822


# Which type of posts receive more comments on average?

Now I have separated ask posts and show posts into different lists, I'll calculate the average number of comments each type of post receives.

In [74]:
total_ask_comments = 0

for row in ask_posts:
    total_ask_comments += int(row[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print("The average number of comments on 'Ask HN' posts are:", avg_ask_comments)
    

The average number of comments on 'Ask HN' posts are: 10.393478498741656


In [77]:
total_show_comments = 0

for row in show_posts:
    total_show_comments += int(row[4])

avg_show_comments = total_show_comments / len(show_posts)
print("The average number of comments on 'Show HN' posts are:", avg_show_comments)

The average number of comments on 'Show HN' posts are: 4.886099625910612


On average, posts asking questions on Hacker News receive more comments than posts showing stuff (10 vs. 4). Since 'Ask HN' posts are more likely to receive comments, I'll focus on those posts for the remainder of this anlysis. 

# Are posts created at a certain timing more likely to attract comments? 

For this section, I will:

1. Calculate the amount of 'Ask HN' posts created during each hour of the day, along with the number of comments 
2. Calculate the average number of comments the posts received by hour

In [97]:
# Calculating the amount of 'Ask HN' posts created during each hour of day and the number of comments received
import datetime as dt

results_list = []

# Appending created datetime and corresponding number of comments to results_list
for row in ask_posts:
    results_list.append(
        [row[6], int(row[4])]
    )
    
counts_by_hour = {} # count of posts by hour
comments_by_hour = {} # count of comments by hour
datetime_format = "%m/%d/%Y %H:%M"

for row in results_list:
    date = row[0]
    c_count = row[1]
    date_dt = dt.datetime.strptime(date, datetime_format)
    hour = dt.datetime.strftime(date_dt,"%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = c_count
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += c_count
    
counts_by_hour

{'02': 269,
 '01': 282,
 '22': 383,
 '21': 518,
 '19': 552,
 '17': 587,
 '15': 646,
 '14': 513,
 '13': 444,
 '11': 312,
 '10': 282,
 '09': 222,
 '07': 226,
 '03': 271,
 '23': 343,
 '20': 510,
 '16': 579,
 '08': 257,
 '00': 301,
 '18': 614,
 '12': 342,
 '04': 243,
 '06': 234,
 '05': 209}

# Average number of comments for 'Ask HN' posts by hour

In [99]:
avg_by_hour = [] # this will store a list(of lists) displaying avg num of comments received by 'Ask HN' posts, by hour

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_by_hour

[['02', 11.137546468401487],
 ['01', 7.407801418439717],
 ['22', 8.804177545691905],
 ['21', 8.687258687258687],
 ['19', 7.163043478260869],
 ['17', 9.449744463373083],
 ['15', 28.676470588235293],
 ['14', 9.692007797270955],
 ['13', 16.31756756756757],
 ['11', 8.96474358974359],
 ['10', 10.684397163120567],
 ['09', 6.653153153153153],
 ['07', 7.013274336283186],
 ['03', 7.948339483394834],
 ['23', 6.696793002915452],
 ['20', 8.749019607843136],
 ['16', 7.713298791018998],
 ['08', 9.190661478599221],
 ['00', 7.5647840531561465],
 ['18', 7.94299674267101],
 ['12', 12.380116959064328],
 ['04', 9.7119341563786],
 ['06', 6.782051282051282],
 ['05', 8.794258373205741]]

The above list is hard to read, so I will:

1. Swap the columns so the avg numbers come first, then the corresponding hours
2. Sort the list in descending order

In [109]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append(
    [row[1], row[0]]
    )

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[28.676470588235293, '15'],
 [16.31756756756757, '13'],
 [12.380116959064328, '12'],
 [11.137546468401487, '02'],
 [10.684397163120567, '10'],
 [9.7119341563786, '04'],
 [9.692007797270955, '14'],
 [9.449744463373083, '17'],
 [9.190661478599221, '08'],
 [8.96474358974359, '11'],
 [8.804177545691905, '22'],
 [8.794258373205741, '05'],
 [8.749019607843136, '20'],
 [8.687258687258687, '21'],
 [7.948339483394834, '03'],
 [7.94299674267101, '18'],
 [7.713298791018998, '16'],
 [7.5647840531561465, '00'],
 [7.407801418439717, '01'],
 [7.163043478260869, '19'],
 [7.013274336283186, '07'],
 [6.782051282051282, '06'],
 [6.696793002915452, '23'],
 [6.653153153153153, '09']]

Next, I will print the top 5 hours that get the highest average comments.

In [126]:
print("Top 5 hours for 'Ask HN' post comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H: %M"),avg
        )
    )

Top 5 hours for 'Ask HN' post comments
15: 00: 28.68 average comments per post
13: 00: 16.32 average comments per post
12: 00: 12.38 average comments per post
02: 00: 11.14 average comments per post
10: 00: 10.68 average comments per post


# Findings

The hour that received the most comments on average is 15:00, with an average of 28.68 comments per post. That's about a 75% increase from the avg number of comments received by the second best hour.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US, which converts to 3:00 AM SGT since I'm based in Singapore. 

# Conclusion

In this project, I analyzed Hacker News posts to determine which type of post and timing receive the most comments on average. 

Based on my analysis, to maximize the amount of comments a post receives, I'd recommend the post be categorized as 'Ask HN' and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set I analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, 'Ask HN' posts received more comments on average and 'Ask HN' posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.